# Quick introduction
The goal of this project is to reduce the patience doctor ratio for Zambian clinics and hospitals. One of the ways in which we can archive this is explore the common issues that patients see a doctor for and attempt to find alternative ways in which those issue can be found. OOne of the the issues topping the list was that of skin infections. In this notebook, will train a model that uses deep lesrning to distingusih betwwen various skin deaseses.

## Undestanding skin infections
There are mulptiple types of skin deases that have varrying implications from death threatening to naive ones that may not require immideate intervention. Severity here ranges from trivial mosquito bites, irritations due to allegies, blister all the way to more complex ones likes STD's and Cellulitis which may require medical emergence upon suspision. 

## Datasets
For the training, we will be using the Skin Disease Dataset from kagle availabe [here](https://www.kaggle.com/data/58249) which consists of over 23000 images of different kinds of skin infections.

We begin by declaring all the imports that will reuired for the traing as indicated below.

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt 
from torchvision import datasets, transforms, models
from PIL import Image
import numpy as np
from collections import OrderedDict
import time
from datetime import datetime

## Loading the data
We load the data from the directory provided and peform image transformations like rotations, horizontal flips etc.. We then load a pre trained model from torchvision and difine a classifer for it.

In [2]:
data_dir = 'Data'
train_dir = data_dir+'/train'
test_dir = data_dir+'/test'

In [3]:
# peform data transafomrations for the training set.
train_transforms = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(244),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

# load the datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# define the dataloader
trainloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=32)
testloader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=32)

In [4]:
# download a pre trained model
model_name = 'resnet152'
model = models.resnet152(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
## Freeze the parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

In [6]:
# modify the classifier to match the dataset
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(2048, 1000)),
    ('Dropout', nn.Dropout(0.7)),
    ('ReLU', nn.ReLU()),
    ('Dropout', nn.Dropout(0.25)),
    ('fc2', nn.Linear(1000, 23)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.fc = classifier

In [7]:
# set the learning ratet
learning_rate = 0.001

In [8]:
# set the optimizer to adam
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

In [9]:
# set the loss function
criterion = nn.NLLLoss()

In [10]:
# set device agnostic code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# declare the variables for ploting
running_loses, test_loses = [], []

In [3]:
model.to(device)
epochs = 50
steps = 0
print_every = 5
starting_time = time.time()

# print a statement to show the start of the training along with the number of epochs
print("########################################################\n")
print("Stating the training at: \n",
      "Time: {}\nEpochs: {}\nDevice: {}\nLearning rate: {}\nArchitecture: {}\n".
      format(datetime.now().strftime('%H:%M:%S'), epochs, device.type, learning_rate, model_name),
             "########################################################")

# start training the model
for e in range(epochs):
    running_loss = 0
    steps+=1
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the gradients
        optimizer.zero_grad()
        
        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    if steps%print_every == 0:
        test_loss = 0
        accuracy = 0
            
        with torch.no_grad():
            model.eval()
                
            for inputs, labels in testloader:
                inputs, lables = inputs.to(device), labels.to(device)
                    
                logps = model(inputs)
                loss = criterion(logps, labels)
                
                test_loss += loss.item()
                    
                ps = torch.exp(logps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
            # append the loses for graphing
            running_loses.append(running_loss/len(trainloader))
            test_loses.append(test_loss/len(testloader))
                    
            print("Epoch: {}/{}".format(e+1,epochs),
                  "Running loss: {:.3f}..".format(running_loss/len(trainloader)),
                  "Test loss: {:.3f}..".format(test_loss/len(testloader)),
                  "Accuracy: {:.3f}..".format(accuracy.item()/len(testloader)*100))
                    
            model.train()
stoping_time = time.time()
duration = stoping_time-starting_time

NameError: name 'model' is not defined

In [1]:
# run this cell if you manully stoped the traning
duration = time.time() - starting_time

NameError: name 'time' is not defined

In [2]:
# a plot of the progress with matplotlib
%matplotlib inline
%config InlineBackend.figure_fromat = 'retina'

plt.plot(running_loses, label='running loses')
plt.plot(test_loses, label='validation loses')
plt.legend(frameon=False)

NameError: name 'plt' is not defined

## Saving and loading trained models
After trainig the model, it is important to save the current state parameters and all other parameters that may help compare the architectures and the hyperparameters.

In [ ]:
checkpoint_dir = 'checkpoints/'

In [ ]:
# crete custom dictionary to save additional params
checkpoint = {'epoch': e,
              'learning_rate': learning_rate,
              'arch': 'cpu',
              'optimizer_state_dict': optimizer.state_dict(),
             'model_state_dict': model.state_dict(),
              'loss': loss,
             'duration': duration}

# save the model
torch.save(checkpoint, checkpoint_dir+model_name+'.pth')

In [ ]:
# load a saved model from disk to coninue traning
checkpoint = torch.load(checkpoint_dir+model_name+'.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']